<a href="https://colab.research.google.com/github/hammaad2002/AdversarialAttack/blob/main/FGSM_wav2vec2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import IPython
import requests
import torch
import torchaudio

In [19]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [20]:
SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"  # noqa: E501
SPEECH_FILE = "_assets/speech.wav"

In [21]:
if not os.path.exists(SPEECH_FILE):
    os.makedirs("_assets", exist_ok=True)
    with open(SPEECH_FILE, "wb") as file:
        file.write(requests.get(SPEECH_URL).content)

In [22]:
IPython.display.Audio(SPEECH_FILE)

In [23]:
waveform, sample_rate = torchaudio.load(SPEECH_FILE)
waveform = waveform.to(device)

In [24]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)

In [25]:
class GreedyCTCDecoder(torch.nn.Module):
    def __init__(self, labels, blank=0):
        super().__init__()
        self.labels = labels
        self.blank = blank

    def forward(self, emission: torch.Tensor) -> str:
        """Given a sequence emission over labels, get the best path string
        Args:
          emission (Tensor): Logit tensors. Shape `[num_seq, num_label]`.

        Returns:
          str: The resulting transcript
        """
        indices = torch.argmax(emission, dim=-1)  # [num_seq,]
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = [i for i in indices if i != self.blank]
        return "".join([self.labels[i] for i in indices])

In [26]:
%%capture
!pip install jiwer

In [27]:
from jiwer import wer

In [28]:
with torch.inference_mode():
    emission, _ = model(waveform)
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
transcript = decoder(emission[0])
print(transcript.replace("|"," "))
IPython.display.Audio(SPEECH_FILE)

I HAD THAT CURIOSITY BESIDE ME AT THIS MOMENT 


In [29]:
import os
import requests
import tarfile

# URL of the mini LibriSpeech dataset
URL = "http://www.openslr.org/resources/31/dev-clean-2.tar.gz"

# Destination directory to save the dataset
DEST_DIR = "/content/LibriSpeech"

# Create the destination directory if it doesn't exist
if not os.path.exists(DEST_DIR):
    os.makedirs(DEST_DIR)

# Download the tarball and save it to a local file
response = requests.get(URL, stream=True)
with open(os.path.join(DEST_DIR, "dev-clean-2.tar.gz"), "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        f.write(chunk)

# Extract the tarball to the destination directory
with tarfile.open(os.path.join(DEST_DIR, "dev-clean-2.tar.gz"), "r:gz") as tar:
    tar.extractall(path=DEST_DIR)

# Remove the tarball file
os.remove(os.path.join(DEST_DIR, "dev-clean-2.tar.gz"))


In [30]:
import os
import json
import glob

root = "/content/LibriSpeech/LibriSpeech/dev-clean-2/"

transcriptions = []

for speaker_id in os.listdir(root):
    speaker_dir = os.path.join(root, speaker_id)
    for chapter_id in os.listdir(speaker_dir):
        chapter_dir = os.path.join(speaker_dir, chapter_id)
        text_files = glob.glob(os.path.join(chapter_dir, "*.txt"))
        if text_files:
            text_file = text_files[0]
            with open(text_file, "r") as f:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split(" ")
                    audio_file = os.path.join(chapter_dir, parts[0] + ".flac")
                    transcription = " ".join(parts[1:])
                    transcriptions.append({
                        "audio_path": audio_file,
                        "transcription": transcription
                    })

with open("transcriptions.json", "w") as f:
    json.dump(transcriptions, f,indent = 4)

In [75]:
import IPython.display as ipd
import random
import librosa
# Play 3 random audio files and display transcription
for i in range(3):
    random_transcription = random.choice(transcriptions)
    audio_path = random_transcription["audio_path"]
    transcription = random_transcription["transcription"]
    audio, sr = librosa.load(audio_path, sr=None)
    ipd.display(ipd.Audio(audio, rate=sr))
    print("Transcription: ", transcription)

Transcription:  BUT MY UNCLE WAS IN NO HUMOR TO WAIT


Transcription:  YES CHARCOAL


Transcription:  THUS ONE SAUNTERS ON AND ON IN THE GLORIOUS RADIANCE IN UTTER PEACE AND FORGETFULNESS OF TIME


In [76]:
audio, sr = librosa.load(audio_path, sr=None)
ipd.display(ipd.Audio(audio, rate=sr))

In [40]:
transcription

'THEN DARKNESS LIKE DEATH'

In [77]:
audio_path

'/content/LibriSpeech/LibriSpeech/dev-clean-2/3000/15664/3000-15664-0007.flac'

In [41]:
def encode_transcription(transcription):
    # Define the dictionary
    dictionary = {'-': 0, '|': 1, 'E': 2, 'T': 3, 'A': 4, 'O': 5, 'N': 6, 'I': 7, 'H': 8, 'S': 9, 'R': 10, 'D': 11, 'L': 12, 'U': 13, 'M': 14, 'W': 15, 'C': 16, 'F': 17, 'G': 18, 'Y': 19, 'P': 20, 'B': 21, 'V': 22, 'K': 23, "'": 24, 'X': 25, 'J': 26, 'Q': 27, 'Z': 28}

    # Convert transcription string to list of characters
    chars = list(transcription)

    # Encode each character using the dictionary
    encoded_chars = [dictionary[char] for char in chars]

    # Concatenate the encoded characters to form the final encoded transcription
    encoded_transcription = torch.tensor(encoded_chars)

    return encoded_transcription

In [44]:
transcription

'THEN DARKNESS LIKE DEATH'

In [45]:
encode_transcription(transcription.replace(" ","|"))

tensor([ 3,  8,  2,  6,  1, 11,  4, 10, 23,  6,  2,  9,  9,  1, 12,  7, 23,  2,
         1, 11,  2,  4,  3,  8])

In [90]:
import torch.nn.functional as F

def fast_gradient_sign_method(model, audio, transcription, epsilon=0.02, device="cpu"):
    data_waveform, sample_rate = torchaudio.load(audio)
    audio = data_waveform.to(device)
    audioo = audio.clone().requires_grad_()
    #freeze model's weights
    model.eval()
    
    # Encode the transcription as integers
    encoded_transcription = encode_transcription(transcription)
    
    # Compute the length of the encoded transcription
    target_lengths = torch.tensor([len(encoded_transcription)], dtype=torch.long)
    
    # Generate adversarial example
    output,_ = model(audioo)
    loss = F.ctc_loss(output.transpose(0, 1), torch.tensor(encoded_transcription, dtype=torch.long),
                      torch.tensor([output.shape[1]], dtype=torch.long), target_lengths)
    loss.backward()
    
    with torch.no_grad():
      #noise_grad = torch.sign(inp_audio.grad.to(device))
        noise_grad = torch.sign(audioo.grad.to(device))
        perturbed_audio = audio + epsilon * noise_grad
        perturbed_audio.detach_()
        #perturbed_audio = torch.clamp(perturbed_audio, -1, 1)
     
    #return original_transcription, perturbed_transcription
    return perturbed_audio

In [91]:
# generate perturbed audio
perturbed_audio = fast_gradient_sign_method(model, audio_path, transcription.replace(" ","|"))
sr = 16000
# define the path where you want to save the perturbed audio
torchaudio.save( "/content/perturbed_audio.flac" , perturbed_audio , sr )

<ipython-input-90-a3ace453a6e6>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = F.ctc_loss(output.transpose(0, 1), torch.tensor(encoded_transcription, dtype=torch.long),


In [92]:
audio_path = '/content/perturbed_audio.flac'
audio, sr = librosa.load(audio_path, sr=None)
ipd.display(ipd.Audio(audio, rate=sr))

In [93]:
waveform, sample_rate = torchaudio.load(audio_path)
waveform = waveform.to(device)

In [96]:
with torch.inference_mode():
    emission, _ = model(waveform)
decoder = GreedyCTCDecoder(labels=bundle.get_labels())
transcript = decoder(emission[0])
print(transcript.replace("|"," "))
print(transcription.replace("|"," "))
audio, sr = librosa.load(audio_path, sr=None)
ipd.display(ipd.Audio(audio, rate=sr))

THUS ONE SAUNTERES ON AND ON IN GLOYOUS GRATEN  IN UTTER PEACE AND FORGETFULNESS OF TIME 
THUS ONE SAUNTERS ON AND ON IN THE GLORIOUS RADIANCE IN UTTER PEACE AND FORGETFULNESS OF TIME


In [32]:
'''import json
import os
import torch
import torchaudio
from tqdm import tqdm
from jiwer import wer

# Load MiniLibriSpeech JSON dataset
with open("/content/transcriptions.json") as f:
    data = json.load(f)

# Calculate WER for MiniLibriSpeech dataset
total_wer = 0
total_words = 0
pred_actual_words = ""

for sample in tqdm(data):
    file_path = sample["audio_path"]
    transcript = sample["transcription"]

    waveform, sample_rate = torchaudio.load(file_path)
    waveform = waveform.to(device)

    with torch.inference_mode():
        emission, _ = model(waveform)

    predicted_transcript = decoder(emission[0]).replace("|", " ")
    predicted_words = predicted_transcript.split()
    actual_words = transcript.split()

    # Check if the lengths of the predicted and actual words match
    if len(predicted_words) == len(actual_words):
        # Calculate WER using jiwer package
        distance = wer(actual_words, predicted_words)
    else:
        # Make the lengths of predicted and actual words same
        if len(predicted_words) > len(actual_words):
            actual_words += ["<eps>"] * (len(predicted_words) - len(actual_words))
        else:
            predicted_words += ["<eps>"] * (len(actual_words) - len(predicted_words))

        # Calculate WER using jiwer package
        distance = wer(actual_words, predicted_words)

    wer_score = distance * 100 / max(len(actual_words), 1)
    total_wer += wer_score
    total_words += len(actual_words)

    # Add predicted and actual words to the file
    pred_actual_words += "Predicted words: " + " ".join(predicted_words) + "\n"
    pred_actual_words += "Actual words   : " + " ".join(actual_words) + "\n"

# Calculate average WER
avg_wer = total_wer / max(total_words, 1)

# Save predicted and actual words and WER to a file
with open("results.txt", "w") as f:
    f.write("WER: {:.2f}%\n\n".format(avg_wer))
    f.write(pred_actual_words)'''

'import json\nimport os\nimport torch\nimport torchaudio\nfrom tqdm import tqdm\nfrom jiwer import wer\n\n# Load MiniLibriSpeech JSON dataset\nwith open("/content/transcriptions.json") as f:\n    data = json.load(f)\n\n# Calculate WER for MiniLibriSpeech dataset\ntotal_wer = 0\ntotal_words = 0\npred_actual_words = ""\n\nfor sample in tqdm(data):\n    file_path = sample["audio_path"]\n    transcript = sample["transcription"]\n\n    waveform, sample_rate = torchaudio.load(file_path)\n    waveform = waveform.to(device)\n\n    with torch.inference_mode():\n        emission, _ = model(waveform)\n\n    predicted_transcript = decoder(emission[0]).replace("|", " ")\n    predicted_words = predicted_transcript.split()\n    actual_words = transcript.split()\n\n    # Check if the lengths of the predicted and actual words match\n    if len(predicted_words) == len(actual_words):\n        # Calculate WER using jiwer package\n        distance = wer(actual_words, predicted_words)\n    else:\n        # 

In [33]:
'''from google.colab import files

# Download the file to your local device
files.download('/content/results.txt')'''

"from google.colab import files\n\n# Download the file to your local device\nfiles.download('/content/results.txt')"